In [1]:
import torch
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, hamming_loss
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from bert_dataset import BertDataset
from preprocessing.utils import is_sentence_in_boundaries
from datasets_utils import get_luxury_data, get_tech_data, get_retail_data, get_big_basket_data
from preprocess import preprocess, with_category_features
from bert import BERT
from bert_dataset import BertDataset
from bert_train import bert_train
from bert_test import bert_test

device = 'cuda' if torch.cuda.is_available() else 'cpu'

[nltk_data] Downloading package stopwords to /home/stepan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/stepan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/stepan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
device

'cuda'

In [3]:
big_basket_data = get_big_basket_data()
# luxury_data = get_luxury_data()
# tech_data = get_tech_data()
# retail_data = get_retail_data()

datasets = [big_basket_data]
dataset_names = ['Big basket']
# datasets = [big_basket_data, retail_data, luxury_data, tech_data]
# dataset_names = ['Big basket', 'Retail', 'Luxury', 'Tech']

# Get datasets with description column preprocessed
big_basket_data['description'] = big_basket_data['description'].apply(preprocess)
# tech_data['description'] = tech_data['description'].apply(preprocess)
# luxury_data['description'] = luxury_data['description'].apply(preprocess)
# retail_data['description'] = retail_data['description'].apply(preprocess)

# Preprocess categories
big_basket_data = with_category_features(big_basket_data)
# tech_data = with_category_features(tech_data)
# luxury_data = with_category_features(luxury_data)
# retail_data = with_category_features(retail_data)

big_basket_data = big_basket_data[big_basket_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=200))]
# retail_data = retail_data[retail_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=250))]
# luxury_data = luxury_data[luxury_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=100))]
# tech_data = tech_data[tech_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=200))]

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[category] = data['category'].apply(lambda x: 1 if category in x else 0)
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[category] = data['category'].apply(lambda x: 1 if category in x else 0)
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [4]:
categories = [column for column in big_basket_data.columns if column != 'description']

big_basket_X_train, big_basket_X_test, big_basket_y_train, big_basket_y_test = train_test_split(
    big_basket_data['description'],
    big_basket_data[categories],
    test_size=0.2,
    random_state=13
)

In [5]:
big_basket_X_train = big_basket_X_train.reset_index()
big_basket_X_train = big_basket_X_train.drop(['index'], axis=1)

big_basket_X_test = big_basket_X_test.reset_index()
big_basket_X_test = big_basket_X_test.drop(['index'], axis=1)

big_basket_y_train = big_basket_y_train.reset_index()
big_basket_y_train = big_basket_y_train.drop(['index'], axis=1)

big_basket_y_test = big_basket_y_test.reset_index()
big_basket_y_test = big_basket_y_test.drop(['index'], axis=1)

In [6]:
big_basket_X_train.shape, big_basket_X_test.shape

((21156, 1), (5289, 1))

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

MAX_LEN = 512
BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-5
NUM_CLASSES = len(big_basket_data.columns) - 1

big_basket_train_dataset = BertDataset(big_basket_X_train, MAX_LEN, tokenizer, big_basket_y_train[categories])
big_basket_test_dataset = BertDataset(big_basket_X_test, MAX_LEN, tokenizer, big_basket_y_test[categories])

big_basket_train_loader = DataLoader(big_basket_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
big_basket_test_loader = DataLoader(big_basket_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
big_basket_bert_model = BERT(NUM_CLASSES)
# big_basket_bert_model.load_state_dict(torch.load('big_basket_bert_weights.bin', map_location=torch.device(device)))
big_basket_bert_model = big_basket_bert_model.to(device)

In [10]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.AdamW(params=big_basket_bert_model.parameters(),
                  lr=LEARNING_RATE)

In [11]:
big_basket_train_result = bert_train(
    model=big_basket_bert_model,
    train_loader=big_basket_train_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=EPOCHS,
    device=device,
)

Batch: 500
Epoch: 0, Loss:  0.10057325661182404
Batch: 1000
Epoch: 0, Loss:  0.07661626487970352
Batch: 1500
Epoch: 0, Loss:  0.06772053241729736
Batch: 2000
Epoch: 0, Loss:  0.07876542210578918
Batch: 2500
Epoch: 0, Loss:  0.09890714287757874
Batch: 3000
Epoch: 0, Loss:  0.07094499468803406
Batch: 3500
Epoch: 0, Loss:  0.06014532968401909
Batch: 4000
Epoch: 0, Loss:  0.057778216898441315
Batch: 4500
Epoch: 0, Loss:  0.0611313059926033
Batch: 5000
Epoch: 0, Loss:  0.06829587370157242


In [13]:
# torch.save(big_basket_bert_model.state_dict(), "big_basket_bert_weights.bin")

In [16]:
torch.tensor(big_basket_train_result['accuracy'][0]).mean()

tensor(0.9799)

In [17]:
big_basket_test_result = bert_test(
    model=big_basket_bert_model,
    validation_loader=big_basket_test_loader,
    loss_fn=loss_fn,
    device=device,
)

Batch: 100
Batch: 200
Batch: 300
Batch: 400
Batch: 500
Batch: 600
Batch: 700
Batch: 800
Batch: 900
Batch: 1000
Batch: 1100
Batch: 1200
Batch: 1300


In [26]:
torch.tensor(big_basket_test_result['accuracy_subset']).mean()

tensor(0.0792)

In [58]:
it = iter(big_basket_test_loader)
# next(it)
# next(it)
# next(it)
test_item = next(it)
input_ids = test_item['ids'].to(device)
attention_mask = test_item['mask'].to(device)
token_type_ids = test_item['token_type_ids'].to(device)
targets = test_item['targets'].cpu().detach().numpy()
output = big_basket_bert_model(input_ids, attention_mask, token_type_ids)
 # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
output = torch.sigmoid(output).cpu().detach().numpy().round()
# thresholding at 0.5
# output = output.flatten().round().numpy()
# np.sum(output==targets)

In [59]:
output

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [60]:
targets

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [70]:
from sklearn.metrics import accuracy_score

accuracy_score(targets, output)

0.25

In [68]:
correct_predictions = np.sum(output==targets)

correct_predictions

412